In [43]:
import pandas as pd
import numpy as np
import os

In [44]:
HistoryHeader = "LOAN SEQUENCE NUMBER,MONTHLY REPORTING PERIOD,CURRENT ACTUAL UPB,CURRENT LOAN DELINQUENCY STATUS,LOAN AGE,REMAINING MONTHS TO LEGAL MATURITY,DEFECT SETTLEMENT DATE,MODIFICATION FLAG,ZERO BALANCE CODE,ZERO BALANCE EFFECTIVE DATE,CURRENT INTEREST RATE,CURRENT DEFERRED UPB,DUE DATE OF LAST PAID INSTALLMENT (DDLPI),MI RECOVERIES,NET SALE PROCEEDS,NON MI RECOVERIES,EXPENSES,LEGAL COSTS,MAINTENANCE AND PRESERVATION COSTS,TAXES AND INSURANCE,MISCELLANEOUS EXPENSES,ACTUAL LOSS CALCULATION,MODIFICATION COST,STEP MODIFICATION FLAG,DEFERRED PAYMENT PLAN,ESTIMATED LOAN TO VALUE (ELTV),ZERO BALANCE REMOVAL UPB,DELINQUENT ACCRUED INTEREST,DELINQUENCY DUE TO DISASTER,BORROWER ASSISTANCE STATUS CODE,CURRENT MONTH MODIFICATION COST,INTEREST BEARING UPB"

In [45]:
HistorySelectedHeader = "LOAN SEQUENCE NUMBER,MONTHLY REPORTING PERIOD,CURRENT ACTUAL UPB," \
"CURRENT LOAN DELINQUENCY STATUS,LOAN AGE,REMAINING MONTHS TO LEGAL MATURITY,MODIFICATION FLAG," \
"CURRENT INTEREST RATE," \
"CURRENT DEFERRED UPB," \
"MODIFICATION COST," \
"STEP MODIFICATION FLAG,DEFERRED PAYMENT PLAN,ESTIMATED LOAN TO VALUE (ELTV)," \
"DELINQUENCY DUE TO DISASTER," \
"BORROWER ASSISTANCE STATUS CODE,CURRENT MONTH MODIFICATION COST,INTEREST BEARING UPB"

In [46]:
Nrows = 100000
dataset_name = "C:\\Users\\Yuxia\\Desktop\\CS-Project\\T-BiLSTM\\code-test\\Data\\historical_data_time_2015Q1.txt"

In [47]:
dataset  = pd.read_csv(dataset_name, sep='|',header=None, names=HistoryHeader.split(","),
                                       index_col=False,nrows=Nrows,low_memory=False)

In [48]:
dataset = dataset[HistorySelectedHeader.split(",")]

In [49]:
# 'CURRENT LOAN DELINQUENCY STATUS' 处理，0，1 和 '0'，'1' 都要替换为 0，其他的替换为 1
# 在这之前不需要变化的有： 编号， 报告月份， 当前欠款
dataset['CURRENT LOAN DELINQUENCY STATUS'] = dataset['CURRENT LOAN DELINQUENCY STATUS'].replace(['0', '1',0, 1], 0)
dataset['CURRENT LOAN DELINQUENCY STATUS'] = dataset['CURRENT LOAN DELINQUENCY STATUS'].apply(lambda x: 1 if x != 0 else 0)

In [50]:
# 在这之前不用变化的是： Legal Maturity Date， 贷款已存续的时间长度
# 新增一列： 贷款存续的时间长度 / 剩余到期月份 + 贷款存续的时间长度， 即： 还款阶段
dataset['REPAYMENT_STAGE'] = dataset['LOAN AGE'] / (dataset['REMAINING MONTHS TO LEGAL MATURITY'] + dataset['LOAN AGE'])

In [51]:
# modification flag 独热向量处理 Y P Null
dataset['MODIFICATION FLAG'] = dataset['MODIFICATION FLAG'].fillna('Null') # 填充空值为 Null
dataset['MODIFICATION FLAG'] = pd.Categorical(
    dataset['MODIFICATION FLAG'], categories=[ 'Null','Y', 'P'], ordered=True) 
dataset = pd.get_dummies(dataset, columns=['MODIFICATION FLAG'], prefix='MODIFICATION_FLAG', drop_first=True) 
# print(dataset['MODIFICATION FLAG'].unique())

In [52]:
# 在这之前不用变化的是：CURRENT DEFERRED UPB
dataset['MODIFICATION COST'] = dataset['MODIFICATION COST'].fillna(0) # 填充空值为 0

In [53]:
# 对 STEP MODIFICATION FLAG 进行独热向量处理， Null Y N
dataset['STEP MODIFICATION FLAG'] = dataset['STEP MODIFICATION FLAG'].fillna('Null')
dataset['STEP MODIFICATION FLAG'] = pd.Categorical(
    dataset['STEP MODIFICATION FLAG'], categories=['Null', 'Y', 'N'], ordered=True)
dataset = pd.get_dummies(dataset, columns=['STEP MODIFICATION FLAG'], prefix='STEP_MODIFICATION_FLAG', drop_first=True)

In [54]:
# 对 DEFERRED PAYMENT PLAN进行独热向量处理, Null P Y
dataset['DEFERRED PAYMENT PLAN'] = dataset['DEFERRED PAYMENT PLAN'].fillna('Null')
dataset['DEFERRED PAYMENT PLAN'] = pd.Categorical(
    dataset['DEFERRED PAYMENT PLAN'], categories=['Null', 'P', 'Y'], ordered=True)
dataset = pd.get_dummies(dataset, columns=['DEFERRED PAYMENT PLAN'], prefix='DEFERRED_PAYMENT_PLAN', drop_first=True)

In [55]:
# 创建ELTV masked 列, 小于2017年四月为false， 其他的为true
dataset['ELTV_MASKED'] = dataset['MONTHLY REPORTING PERIOD'].apply(lambda x: 1 if x >= 201704 else 0)

In [56]:
# 处理ELTV
dataset['ESTIMATED LOAN TO VALUE (ELTV)'] = dataset['ESTIMATED LOAN TO VALUE (ELTV)'].fillna(0) # 填充空值为 0

In [57]:
# 处理 'DELINQUENCY DUE TO DISASTER' ,null 改为false, 其他的改为true
dataset['DELINQUENCY DUE TO DISASTER'] = dataset['DELINQUENCY DUE TO DISASTER'].fillna('Null')
dataset['DELINQUENCY DUE TO DISASTER'] = pd.Categorical(
    dataset['DELINQUENCY DUE TO DISASTER'], categories=['Null', 'Y'], ordered=True)
dataset = pd.get_dummies(dataset, columns=['DELINQUENCY DUE TO DISASTER'], prefix='DELINQUENCY_DUE_TO_DISASTER', drop_first=True)

In [58]:
# 处理 'Borrowed Assistance Status Code'
dataset['BORROWER ASSISTANCE STATUS CODE'] = dataset['BORROWER ASSISTANCE STATUS CODE'].fillna('Null')
dataset['BORROWER ASSISTANCE STATUS CODE'] = pd.Categorical(
    dataset['BORROWER ASSISTANCE STATUS CODE'], categories=['Null', 'F', 'R','T'], ordered=True)
dataset = pd.get_dummies(dataset, columns=['BORROWER ASSISTANCE STATUS CODE'], prefix='BORROWER_ASSISTANCE_STATUS_CODE', drop_first=True)


In [59]:
# 处理 current month modification cost
dataset['CURRENT MONTH MODIFICATION COST'] = dataset['CURRENT MONTH MODIFICATION COST'].fillna(0) # 填充空值为 0

In [60]:
# 处理INTEREST BEARING UPB （分为0和非0）
# dataset['INTEREST BEARING UPB'] = dataset['INTEREST BEARING UPB'].apply(lambda x: 0.0 if x == 0.0 else 1.0)
# dataset['INTEREST BEARING UPB'] = pd.Categorical(
#     dataset['INTEREST BEARING UPB'], categories=[0.0, 1.0], ordered=True)
# dataset = pd.get_dummies(dataset, columns=['INTEREST BEARING UPB'], prefix='INTEREST_BEARING_UPB', drop_first=True)

In [61]:
# 处理每月还款
dataset['CURRENT ACTUAL UPB-Delta'] = dataset.groupby('LOAN SEQUENCE NUMBER')['CURRENT ACTUAL UPB'].diff()
dataset['CURRENT ACTUAL UPB-Delta'] = dataset['CURRENT ACTUAL UPB-Delta'].fillna(0)
dataset['CURRENT ACTUAL UPB-Delta'] = dataset['CURRENT ACTUAL UPB-Delta'].abs()

In [62]:
# INTEREST BEARING UPB
dataset['INTEREST BEARING UPB'] = dataset['INTEREST BEARING UPB'].astype('float32')
dataset['INTEREST BEARING UPB-Delta'] = dataset.groupby('LOAN SEQUENCE NUMBER')['INTEREST BEARING UPB'].diff()
dataset['INTEREST BEARING UPB-Delta'] = dataset['INTEREST BEARING UPB-Delta'].fillna(0)
dataset['INTEREST BEARING UPB-Delta'] = dataset['INTEREST BEARING UPB-Delta'].abs()
# dataset.drop('INTEREST BEARING UPB', axis=1, inplace=True)

In [63]:
dataset.to_csv("C:\\Users\\Yuxia\\Desktop\\CS-Project\\T-BiLSTM\\code-test\\Project_VStart\\data\\test\\encoded_MonthlyData.csv", index=False, header=True)

In [66]:
dataset_show = dataset[['LOAN SEQUENCE NUMBER','CURRENT ACTUAL UPB-Delta', 'INTEREST BEARING UPB-Delta','CURRENT LOAN DELINQUENCY STATUS']]

In [64]:
print(dataset.columns)

Index(['LOAN SEQUENCE NUMBER', 'MONTHLY REPORTING PERIOD',
       'CURRENT ACTUAL UPB', 'CURRENT LOAN DELINQUENCY STATUS', 'LOAN AGE',
       'REMAINING MONTHS TO LEGAL MATURITY', 'CURRENT INTEREST RATE',
       'CURRENT DEFERRED UPB', 'MODIFICATION COST',
       'ESTIMATED LOAN TO VALUE (ELTV)', 'CURRENT MONTH MODIFICATION COST',
       'INTEREST BEARING UPB', 'REPAYMENT_STAGE', 'MODIFICATION_FLAG_Y',
       'MODIFICATION_FLAG_P', 'STEP_MODIFICATION_FLAG_Y',
       'STEP_MODIFICATION_FLAG_N', 'DEFERRED_PAYMENT_PLAN_P',
       'DEFERRED_PAYMENT_PLAN_Y', 'ELTV_MASKED',
       'DELINQUENCY_DUE_TO_DISASTER_Y', 'BORROWER_ASSISTANCE_STATUS_CODE_F',
       'BORROWER_ASSISTANCE_STATUS_CODE_R',
       'BORROWER_ASSISTANCE_STATUS_CODE_T', 'CURRENT ACTUAL UPB-Delta',
       'INTEREST BEARING UPB-Delta'],
      dtype='object')
